In [1]:
import torch
import torchmetrics

In [2]:
import pandas as pd
import os
from sampo.scheduler.selection.neural_net import NeuralNetTrainer, NeuralNet, NeuralNetType
from sklearn.preprocessing import StandardScaler

path = os.path.join(os.getcwd(), 'datasets/dataset_1000_objs.csv')
train_dataset = pd.read_csv(path, index_col='index')
for col in train_dataset.columns[:-1]:
    train_dataset[col] = train_dataset[col].apply(lambda x: float(x))

In [3]:
input_parameters = 13
layer_size = 15
layer_count = 6
classification_size = 2
learning_rate = 0.007

model = NeuralNet(input_parameters, layer_size, layer_count, classification_size, task_type=NeuralNetType.CLASSIFICATION)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scorer = torchmetrics.classification.BinaryAccuracy()

trainer = NeuralNetTrainer(model, criterion, optimizer, scorer, 32)

In [4]:
type(train_dataset)

pandas.core.frame.DataFrame

In [5]:
scaler = StandardScaler()
scaler.fit(train_dataset.drop(columns=['label']))
scaled_dataset = scaler.transform(train_dataset.drop(columns=['label']))
scaled_dataset = pd.DataFrame(scaled_dataset, columns=train_dataset.drop(columns=['label']).columns)
train_dataset = pd.concat([scaled_dataset, train_dataset['label']], axis=1)
train_dataset

,0,1,2,3,4,5,6,7,8,9,10,11,12,label
0,-0.674765,-0.707314,-0.705222,-0.629762,0.204466,0.050051,-0.180452,-0.139112,-0.035459,0.159406,0.057477,1.056521,1.182540,0
1,-0.195432,0.135196,-0.871310,0.261822,0.204466,-0.216586,-0.260307,-0.228194,-0.049212,0.508953,-0.313972,0.132240,0.623127,0
2,-0.445519,0.394123,-0.579806,0.315952,0.204466,-0.090330,-0.222494,-0.186013,-0.064573,0.766199,-0.815762,0.569897,0.888015,0
3,-0.487200,-0.178382,-0.608399,-0.467579,0.204466,-0.066770,-0.215438,-0.178141,0.103124,-0.111545,0.354241,0.651569,0.937446,0
4,0.033814,-0.693454,-0.821721,-0.815304,0.204466,-0.313574,-0.289354,-0.260597,-0.019844,0.264828,0.213477,-0.203965,0.419642,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.471466,-0.325083,-0.306984,-0.274259,0.204466,-0.462384,-0.333920,-0.198254,0.363286,0.040553,-0.079189,0.442881,0.107436,1
1996,0.283901,1.165656,-1.618302,1.200351,2.776360,-0.403578,4.627703,4.690855,-1.686755,-3.311491,-2.058166,-4.207862,-2.003678,1
1997,0.033814,-0.434802,-0.753341,-0.527014,0.204466,-0.313574,-0.289354,-0.260597,0.032857,0.711325,-0.638389,-0.203965,0.419642,1
1998,0.742393,-0.216432,-0.010487,-0.173414,0.204466,-0.536708,-0.356180,-0.335144,-0.563019,0.419580,0.996506,0.099360,-0.048499,1


In [6]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_dataset.drop(columns=['label']), train_dataset['label'])

In [7]:
from sampo.scheduler.selection.neural_net import NeuralNetType
from sampo.scheduler.selection.validation import cross_val_score
from sklearn.metrics import accuracy_score

s = cross_val_score(X=x_train,
                    y=y_train,
                    model=trainer,
                    epochs=100,
                    folds=5,
                    shuffle=True,
                    type_task=NeuralNetType.CLASSIFICATION)

Epoch [1/100], Average Loss: 0.6847
Epoch [2/100], Average Loss: 0.6724
Epoch [3/100], Average Loss: 0.6708
Epoch [4/100], Average Loss: 0.6712
Epoch [5/100], Average Loss: 0.6740
Epoch [6/100], Average Loss: 0.6700
Epoch [7/100], Average Loss: 0.6688
Epoch [8/100], Average Loss: 0.6698
Epoch [9/100], Average Loss: 0.6697
Epoch [10/100], Average Loss: 0.6680
Epoch [11/100], Average Loss: 0.6686
Epoch [12/100], Average Loss: 0.6685
Epoch [13/100], Average Loss: 0.6675
Epoch [14/100], Average Loss: 0.6671
Epoch [15/100], Average Loss: 0.6680
Epoch [16/100], Average Loss: 0.6693
Epoch [17/100], Average Loss: 0.6670
Epoch [18/100], Average Loss: 0.6686
Epoch [19/100], Average Loss: 0.6685
Epoch [20/100], Average Loss: 0.6669
Epoch [21/100], Average Loss: 0.6671
Epoch [22/100], Average Loss: 0.6675
Epoch [23/100], Average Loss: 0.6667
Epoch [24/100], Average Loss: 0.6681
Epoch [25/100], Average Loss: 0.6668
Epoch [26/100], Average Loss: 0.6685
Epoch [27/100], Average Loss: 0.6676
Epoch [28/

2023-10-12 17:11:28,368	WARNING session.py:100 -- In neither tune session nor train session!


Epoch [100/100], Average Loss: 0.6677
Epoch [1/100], Average Loss: 0.6677
Epoch [2/100], Average Loss: 0.6680


C:\SAMPO\venv\lib\site-packages\ray\air\session.py:28: UserWarning: `report` is meant to only be called inside a function that is executed by a Tuner or Trainer. Returning `None`.
  warnings.warn(


Epoch [3/100], Average Loss: 0.6667
Epoch [4/100], Average Loss: 0.6675
Epoch [5/100], Average Loss: 0.6672
Epoch [6/100], Average Loss: 0.6694
Epoch [7/100], Average Loss: 0.6677
Epoch [8/100], Average Loss: 0.6692
Epoch [9/100], Average Loss: 0.6690
Epoch [10/100], Average Loss: 0.6670
Epoch [11/100], Average Loss: 0.6672
Epoch [12/100], Average Loss: 0.6668
Epoch [13/100], Average Loss: 0.6668
Epoch [14/100], Average Loss: 0.6674
Epoch [15/100], Average Loss: 0.6689
Epoch [16/100], Average Loss: 0.6669
Epoch [17/100], Average Loss: 0.6672
Epoch [18/100], Average Loss: 0.6672
Epoch [19/100], Average Loss: 0.6684
Epoch [20/100], Average Loss: 0.6671
Epoch [21/100], Average Loss: 0.6679
Epoch [22/100], Average Loss: 0.6687
Epoch [23/100], Average Loss: 0.6668
Epoch [24/100], Average Loss: 0.6678
Epoch [25/100], Average Loss: 0.6667
Epoch [26/100], Average Loss: 0.6691
Epoch [27/100], Average Loss: 0.6667
Epoch [28/100], Average Loss: 0.6665
Epoch [29/100], Average Loss: 0.6674
Epoch [3

...

Epoch [98/100], Average Loss: 0.6661
Epoch [99/100], Average Loss: 0.6654
Epoch [100/100], Average Loss: 0.6656
Epoch [1/100], Average Loss: 0.6657
Epoch [2/100], Average Loss: 0.6656
Epoch [3/100], Average Loss: 0.6656
Epoch [4/100], Average Loss: 0.6661
Epoch [5/100], Average Loss: 0.6668
Epoch [6/100], Average Loss: 0.6667
Epoch [7/100], Average Loss: 0.6656
Epoch [8/100], Average Loss: 0.6664
Epoch [9/100], Average Loss: 0.6659
Epoch [10/100], Average Loss: 0.6662
Epoch [11/100], Average Loss: 0.6665
Epoch [12/100], Average Loss: 0.6664
Epoch [13/100], Average Loss: 0.6659
Epoch [14/100], Average Loss: 0.6655
Epoch [15/100], Average Loss: 0.6681
Epoch [16/100], Average Loss: 0.6661
Epoch [17/100], Average Loss: 0.6665
Epoch [18/100], Average Loss: 0.6657
Epoch [19/100], Average Loss: 0.6669
Epoch [20/100], Average Loss: 0.6659
Epoch [21/100], Average Loss: 0.6659
Epoch [22/100], Average Loss: 0.6666
Epoch [23/100], Average Loss: 0.6675
Epoch [24/100], Average Loss: 0.6658
Epoch [25

2023-10-12 17:11:54,513	WARNING session.py:100 -- In neither tune session nor train session!


Epoch [100/100], Average Loss: 0.6656
Epoch [1/100], Average Loss: 0.6669
Epoch [2/100], Average Loss: 0.6657
Epoch [3/100], Average Loss: 0.6657
Epoch [4/100], Average Loss: 0.6659
Epoch [5/100], Average Loss: 0.6663
Epoch [6/100], Average Loss: 0.6657
Epoch [7/100], Average Loss: 0.6650
Epoch [8/100], Average Loss: 0.6651
Epoch [9/100], Average Loss: 0.6659
Epoch [10/100], Average Loss: 0.6650
Epoch [11/100], Average Loss: 0.6654
Epoch [12/100], Average Loss: 0.6656
Epoch [13/100], Average Loss: 0.6648
Epoch [14/100], Average Loss: 0.6661
Epoch [15/100], Average Loss: 0.6655
Epoch [16/100], Average Loss: 0.6647
Epoch [17/100], Average Loss: 0.6661
Epoch [18/100], Average Loss: 0.6653
Epoch [19/100], Average Loss: 0.6663
Epoch [20/100], Average Loss: 0.6650
Epoch [21/100], Average Loss: 0.6657
Epoch [22/100], Average Loss: 0.6653
Epoch [23/100], Average Loss: 0.6657
Epoch [24/100], Average Loss: 0.6650
Epoch [25/100], Average Loss: 0.6658
Epoch [26/100], Average Loss: 0.6654
Epoch [27

2023-10-12 17:12:03,222	WARNING session.py:100 -- In neither tune session nor train session!


Epoch [100/100], Average Loss: 0.6655


In [8]:
index = [-10000] * input_parameters
for fold in list(model.parameters())[0]:
    for i in range(len(fold)):
        index[i] = max(index[i], fold[i])
for i in range(len(index)):
    print(i, index[i])

0 tensor(2.3059, grad_fn=<SelectBackward0>)
1 tensor(1.9788, grad_fn=<SelectBackward0>)
2 tensor(2.8585, grad_fn=<SelectBackward0>)
3 tensor(1.6799, grad_fn=<SelectBackward0>)
4 tensor(1.1946, grad_fn=<SelectBackward0>)
5 tensor(1.8484, grad_fn=<SelectBackward0>)
6 tensor(0.6657, grad_fn=<SelectBackward0>)
7 tensor(0.7560, grad_fn=<SelectBackward0>)
8 tensor(1.5524, grad_fn=<SelectBackward0>)
9 tensor(1.2578, grad_fn=<SelectBackward0>)
10 tensor(1.6093, grad_fn=<SelectBackward0>)
11 tensor(1.8630, grad_fn=<SelectBackward0>)
12 tensor(0.4621, grad_fn=<SelectBackward0>)


In [9]:
print(s)

(0.5208333313465119, 0.6621743083000183, <sampo.scheduler.selection.neural_net.NeuralNetTrainer object at 0x000001E393FC6AD0>)


In [10]:
test_dataset = x_test

In [11]:
test_dataset

,0,1,2,3,4,5,6,7,8,9,10,11,12
1963,0.617350,-0.290791,0.350391,-0.294693,0.204466,-0.503817,-0.346329,-0.324156,-0.366428,1.065895,-0.476924,1.366191,0.020507
983,-0.320476,-1.354054,-0.810233,-1.424877,0.204466,-0.156488,-0.242308,-0.208116,0.003344,0.124182,0.199847,0.340565,0.749214
69,-2.133605,3.784072,2.115015,3.891530,-2.367429,2.847600,0.657385,0.314857,2.138104,-1.485907,-1.704467,0.779448,-2.003678
1211,0.408944,-0.199342,0.364670,-0.361685,0.204466,-0.443517,-0.328270,-0.304010,-0.660814,1.095033,-0.126086,0.530083,0.147019
1370,0.888277,-0.835995,-0.047870,-0.792034,0.204466,-0.572355,-0.366856,-0.347054,-0.755636,0.837893,0.732769,-0.065397,-0.123287
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1684,1.492653,-1.037543,-0.372736,-1.035875,0.204466,-0.694957,-0.403574,-0.388014,-0.802895,0.450167,1.428771,-0.632055,-0.380511
1771,1.263407,-1.242496,0.199215,-1.012485,0.204466,-0.652617,-0.390893,-0.373869,-1.013813,0.422268,1.887072,-0.436364,-0.291680
418,1.180045,-0.840916,-0.311822,-0.633079,0.204466,-0.636049,-0.385931,-0.368333,-0.681945,0.539356,1.148464,-0.359785,-0.256919
1560,1.034161,-0.480303,0.632971,-0.562725,0.204466,-0.605375,-0.376745,-0.358086,-0.697739,0.975110,0.276246,-0.218014,-0.192565


In [15]:
predicted = trainer.predict([torch.Tensor(v) for v in test_dataset.iloc[0:2, :].values])
predicted

array([1., 1.], dtype=float32)

In [19]:
type(predicted[1])

numpy.float32

In [15]:
array = []
label_test = y_test.to_numpy()
for i in range(len(predicted)):
    array.append(int(predicted[i] == label_test[i]))
sum(array) / len(array)

TypeError: only size-1 arrays can be converted to Python scalars

In [57]:
predicted

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.